In [1]:
# importando bibliotecas para todos os algoritmos

import pickle
import warnings
import numpy as np 
import pylab as pyl 
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

%pylab inline
%matplotlib inline 

Populating the interactive namespace from numpy and matplotlib


In [2]:
warnings.filterwarnings('ignore')

In [3]:
# importando o dataset
df = pd.read_csv('../dataset/data-set-clube-new-v2.csv', sep=';')

In [4]:
df.head(1)

,id_socio,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
0,38810,CANCELADO,24,0,0,0


In [6]:
df.tail(1)

,id_socio,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
3026,49083,ATIVO,51,4,12,15


In [7]:
# Removendo a coluna id_socio (não usaremos ela por enquanto)
df.drop('id_socio', inplace=True, axis=1)

In [8]:
# Verificando se existem valores nulos
df.isnull().values.any()

False

Após o tratamento abaixo, o STATUS ficará: 

- ATIVO = 0 
- CANCELADO = 1

In [9]:
# Passando a coluna STATUS para o tipo inteiro
label_encoder = preprocessing.LabelEncoder()
df['status'] = label_encoder.fit_transform(df['status'])

In [10]:
# Definindo as classes
status_map = {True : 1, False : 0}

In [11]:
# Aplicando o mapeamento ao dataset
df['status'] = df['status'].map(status_map)

In [12]:
df.head(1)

,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
0,1,24,0,0,0


In [13]:
df.tail(1)

,status,qtde_em_aberto,qtde_em_dia,qtde_em_atraso,qtde_frequencia_ano
3026,0,51,4,12,15


In [14]:
# Verificando como os dados estão distribuídos
num_true = len(df.loc[df['status'] == True])
num_false = len(df.loc[df['status'] == False])
print("Número de Status Cancelados: {0} ({1:2.2f}%)".format(num_true, (num_true/ (num_true + num_false)) * 100))
print("Número de Status Ativos    : {0} ({1:2.2f}%)".format(num_false, (num_false/ (num_true + num_false)) * 100))

Número de Status Cancelados: 771 (25.47%)
Número de Status Ativos    : 2256 (74.53%)


## Spliting

70% para dados de treino e 30% para dados de teste

In [15]:
# Seleção de variáveis preditoras (Feature Selection)
atributos = ['qtde_em_aberto', 'qtde_em_dia', 'qtde_em_atraso', 'qtde_frequencia_ano']

In [16]:
# Variável a ser prevista
atrib_prev = ['status']

In [17]:
# Criando objetos
X = df[atributos].values
Y = df[atrib_prev].values

In [18]:
# Definindo a taxa de split
split_test_size = 0.30

In [19]:
# Criando dados de treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size = split_test_size, random_state = 42)

In [20]:
# Imprimindo os resultados
print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(df.index)) * 100))
print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(df.index)) * 100))

69.97% nos dados de treino
30.03% nos dados de teste


### Verificando o Split

In [21]:
print("Original True : {0} ({1:0.2f}%)".format(len(df.loc[df['status'] == 1]), 
                                               (len(df.loc[df['status'] ==1])/len(df.index) * 100)))

print("Original False : {0} ({1:0.2f}%)".format(len(df.loc[df['status'] == 0]), 
                                               (len(df.loc[df['status'] == 0])/len(df.index) * 100)))
print("")
print("Training True : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 1]), 
                                               (len(Y_treino[Y_treino[:] == 1])/len(Y_treino) * 100)))

print("Training False : {0} ({1:0.2f}%)".format(len(Y_treino[Y_treino[:] == 0]), 
                                               (len(Y_treino[Y_treino[:] == 0])/len(Y_treino) * 100)))
print("")
print("Test True : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 1]), 
                                               (len(Y_teste[Y_teste[:] == 1])/len(Y_teste) * 100)))

print("Test False : {0} ({1:0.2f}%)".format(len(Y_teste[Y_teste[:] == 0]), 
                                               (len(Y_teste[Y_teste[:] == 0])/len(Y_teste) * 100)))

Original True : 771 (25.47%)
Original False : 2256 (74.53%)

Training True : 530 (25.02%)
Training False : 1588 (74.98%)

Test True : 241 (26.51%)
Test False : 668 (73.49%)


## 1 - Naive Bayes - GaussianNB

In [22]:
# Criando o modelo preditivo
modelo_GaussianNB = GaussianNB()

In [23]:
# Treinando o modelo

# ravel "ajusta o shape" do treino pra treinar

modelo_GaussianNB.fit(X_treino, Y_treino.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

##### Verificando a exatidão no modelo nos dados de treino

In [24]:
# Fazendo as previsões somente com a base de teste e guardando em uma variável

gaussian_nb_predict_train = modelo_GaussianNB.predict(X_treino)

In [26]:
# Informa para o accuracy_score o Y_treino como o esperado, para fazer a comparação com a variável
# acima gaussian_nb_predict_train

print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, gaussian_nb_predict_train)))
print()

# Abaixo está o resultado: a cada 100, está acertando 64

Exatidão (Accuracy): 0.6402



##### Verificando a exatidão no modelo nos dados de teste

In [27]:
gaussian_nb_predict_test = modelo_GaussianNB.predict(X_teste)

In [28]:
# Informa para o accuracy_score o Y_test como o esperado, para fazer a comparação com a variável
# acima nb_predict_test
# Diferente do modelo acima (nb_predict_train) esse bloco não conhece a base de treino (X_teste)

print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, gaussian_nb_predict_test)))
print()

# Abaixo está o resultado: a cada 100, está acertando 64

Exatidão (Accuracy): 0.6469



In [30]:
# Criando uma Confusion Matrix
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(Y_teste, gaussian_nb_predict_test, labels = [1, 0])))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, gaussian_nb_predict_test, labels = [1, 0]))

Confusion Matrix
[[210  31]
 [290 378]]

Classification Report
              precision    recall  f1-score   support

           1       0.42      0.87      0.57       241
           0       0.92      0.57      0.70       668

   micro avg       0.65      0.65      0.65       909
   macro avg       0.67      0.72      0.63       909
weighted avg       0.79      0.65      0.67       909



## 2 - RandomForest

In [31]:
# Treinando o modelo

# ravel "ajusta o shape" do treino pra treinar

modelo_RFClassifier = RandomForestClassifier(random_state = 42)
modelo_RFClassifier.fit(X_treino, Y_treino.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [33]:
# Verificando os dados de treino

# Boas chances de conseguir o Overfitting = resultados não muito precisos, pois o Algoritmo pode manter vícios e "lixos"
# nos resultados.
randomf_predict_train = modelo_RFClassifier.predict(X_treino)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_treino, randomf_predict_train)))

Exatidão (Accuracy): 0.9160


In [35]:
# Verificando nos dados de teste

# Esse bloco realmente pode ser levado em consideração
# Aqui apresentamos dados que  o algoritmo nunca viu
randomf_predict_test = modelo_RFClassifier.predict(X_teste)
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, randomf_predict_test)))
print()

Exatidão (Accuracy): 0.8603



In [36]:
print("Confusion Matrix")

print("{0}".format(metrics.confusion_matrix(Y_teste, randomf_predict_test, labels = [1, 0])))
print("")

print("Classification Report")
print(metrics.classification_report(Y_teste, randomf_predict_test, labels = [1, 0]))

# Abaixo a precisão está num total de 87%

Confusion Matrix
[[193  48]
 [ 79 589]]

Classification Report
              precision    recall  f1-score   support

           1       0.71      0.80      0.75       241
           0       0.92      0.88      0.90       668

   micro avg       0.86      0.86      0.86       909
   macro avg       0.82      0.84      0.83       909
weighted avg       0.87      0.86      0.86       909



## 3 - Regressão Logística

In [37]:
# Terceira versão do modelo usando Regressão Logística
modelo_regressaoLog = LogisticRegression(C = 0.7, random_state = 42)
modelo_regressaoLog.fit(X_treino, Y_treino.ravel())

# A avaliação deve ser feita SEMPRE na base de TESTE

regressao_log_predict_test = modelo_regressaoLog.predict(X_teste)

In [47]:
print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(Y_teste, regressao_log_predict_test)))
print()
print("Classification Report")
print(metrics.classification_report(Y_teste, regressao_log_predict_test, labels = [1, 0]))

# Abaixo a precisão está num total de 83%

Exatidão (Accuracy): 0.8383

Classification Report
              precision    recall  f1-score   support

           1       0.69      0.72      0.70       241
           0       0.90      0.88      0.89       668

   micro avg       0.84      0.84      0.84       909
   macro avg       0.79      0.80      0.80       909
weighted avg       0.84      0.84      0.84       909



#### Fazendo Previsões Com o Modelo Treinado

In [45]:
# Salvando o modelo para usar mais tarde
filename = 'Saida_Previsao_Modelo_de_Treino_Regressao.sav'
pickle.dump(modelo_regressaoLog, open(filename, 'wb'))

In [46]:
# Carregando o modelo e fazendo previsão com novos conjuntos de dados 
# (X_teste, Y_teste devem ser novos conjuntos de dados preparados com o procedimento de limpeza e transformação adequados)
loaded_model = pickle.load(open(filename, 'rb'))
resultado1 = loaded_model.predict(X_teste[15].reshape(1, -1))
resultado2 = loaded_model.predict(X_teste[18].reshape(1, -1))
print(resultado1)
print(resultado2)

[0]
[0]
